<a id="top"></a>
# Brain Tumor Segmentation Sample Application using the Intel® Distribution of OpenVINO™ toolkit 

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "brain_tumor_segmentation.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **argparser.py** - Shared Python* code to perform command line argument parsing
    - **brain_tumor_segmentation.ipynb** - This Jupyter* Notebook
    - **brain_tumor_segmentation.py** - Python* code for brain tumor segmentation application
    - **images/*.png** - Images used in this Jupyter* Notebook
    - **settings.py** - Shared Python* code containing configuration settings
    - **/data/Healthcare_app/data/saved_model_frozen.pb** - TensorFlow* model trained to segment brain tumors 
    - **/data/Healthcare_app/data/144x144/Task01_BrainTumour.h5** - Raw MRI input data

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform brain tumor segmentation.  This solution segments brain tumors from raw MRI scans.  The result is a colorized overlay of the original MRI scan highlighting brain tumor matter.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


### Application background

To perform brain tumor segmentation (BraTS) the U-Net architecture is used to segment brain tumors from raw MRI scans. With relatively little data a U-Net model can be trained to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for our model is about 0.82-0.88.  [Menze et al. reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made. The below MRI brain scans show highlighted brain tumor matter segmented using a trained model. 

![](images/figure1.png)

### What is U-Net?
Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

The architecture of U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the `U` shape.  However, in contrast to the autoencoder, U-Net predicts a pixel-wise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

![](images/unet.png)

## Brain tumor segmentation application
The brain tumor segmentation application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on a raw MRI input image to segment brain tumor matter within the image.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the brain tumor segmentation application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the supplied raw MRI images to segment brain tumors.  The result is a combined image comprising a set of three images: The MRI input, ground truth, and the predicted output from inference with Dice score.  The combined image is also annotated with the amount of time to do inference.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU and Intel® Movidius™ Neural Compute Stick 2.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `outputHTML` image display utility.

The application and inference code for this sample is already implemented in the three Python* files: [`healthcare_openvino.py`](./healthcare_openvino.py) and supporting [`argparser.py`](./argparser.py) and [`settings.py`](./settings.py).

The following sections will guide you through configuring and running the brain tumor segmentation application.

### Configuration
The following sections describe all the necessary configuration to run the brain tumor segmentation application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 healthcare_openvino.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`healthcare_openvino.py`](./healthcare_openvino.py) are:
- **-IR** - The precision to complete the path to the .xml IR file (created using Model Optimizer) for the inference model. The file is found using the path models/<precision>/saved_model.xml
- **-r** - The path to where the output image files will be stored
- **-d** - Device type to use to run inference (CPU, GPU, MYRIAD)

### Imports
We begin by importing all the Python* modules that will be used within this Jupyter* Notebook to run and display the results of the brain tumor segmentation application on the Intel® DevCloud for the Edge:
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying results and managing jobs from within this Jupyter* Notebook

Run the following cell to import the Python* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import matplotlib.pyplot as plt
import os
import time
import sys
from qarpo.demoutils import *
print('Imported Python modules successfully.')

In [ ]:
try: 
    import psutil
except:
    print("psutil not installed")
    !{sys.executable} -m pip install psutil

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.).  For this sample, the model is supplied in the file `/data/Healthcare_app/data/saved_model_frozen.pb`.

The `saved_model_frozen.pb` file will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

For this model, we will run the Model Optimizer using the format:
```bash
mo.py \
    --input_model <path_to_model> \
    --input_shape=[<N,H,W,C>] \
    --data_type <data_precision> \
    --output_dir <path_to_output_directory> \
    --model_name <name>
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--input_shape** : The model's input data shape
 - **N** = Batch size
 - **H** = Height
 - **W** = Width
 - **C** = Number of channels
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--output_dir** : Output directory where to store the generated IR model files
- **--model_name** : Output model name to use

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type <data_precision> \
    --output_dir models/<data_precision> \
    --model_name saved_model
```
We will run the command twice, first with <*data_precision*> set to `FP16` and then `FP32` to get all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert TensorFlow* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a TensorFlow* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html)
</i></div>

Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type FP16  \
    --output_dir models/FP16  \
    --model_name saved_model

# Create FP32 IR files
!mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type FP32  \
    --output_dir models/FP32  \
    --model_name saved_model

# find all resulting IR files
!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
healthcare_openvino_job.sh <output_directory> <device> <fp_precision>
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Choose the appropriate inference model IR file for the specified <*fp_precision*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `healthcare_openvino_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `healthcare_openvino_job.sh`.

In [ ]:
%%writefile brain_tumor_seg_job.sh

# Store input arguments: <output_directory> <device> <fp_precision>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=models/${FP_MODEL}/saved_model

# Run the brain tumor segmentation code
python3 brain_tumor_segmentation.py  -IR $MODELPATH \
                                     -r $OUTPUT_FILE \
                                     -d $DEVICE


#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit healthcare_openvino_job.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 
<br><div class=note><i><b>Note: </b>We will not be running this sample on an [Intel® Atom® x7-E3950](https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz.html).  The CPU does not support the AVX instruction set which is enabled by default within the Python* `tensorflow` module libraries used.
</i></div>

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub brain_tumor_seg_job.sh -l nodes=1:idc001skl -F "results/core/ CPU FP32" -N brain_ts_core
print(job_id_core[0])
#Progress indicators
if job_id_core:
    progressIndicator('results/core', f'i_progress_{job_id_core[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_cascade_lake = !qsub brain_tumor_seg_job.sh  -l nodes=1:idc018 -F "results/xeon_cascade_lake/ CPU FP32" -N brain_ts_xeon_cascade_lake 
print(job_id_xeon_cascade_lake[0])
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('results/xeon_cascade_lake', f'i_progress_{job_id_xeon_cascade_lake[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® E3-1268L v5 CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_skylake = !qsub brain_tumor_seg_job.sh  -l nodes=1:idc007xv5 -F "results/xeon_skylake/ CPU FP32" -N brain_ts_xeon_skylake 
print(job_id_xeon_skylake[0])
#Progress indicators
if job_id_xeon_skylake:
    progressIndicator('results/xeon_skylake', f'i_progress_{job_id_xeon_skylake[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Core CPU and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on the Intel® HD Graphics 530 GPU integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub brain_tumor_seg_job.sh -l nodes=1:idc001skl -F "results/gpu/ GPU FP32" -N brain_ts_gpu 
print(job_id_gpu[0])
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', f'i_progress_{job_id_gpu[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Neural Compute Stick 2
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) CPU. The inference workload will run on an [Intel® Neural Compute Stick 2](https://software.intel.com/en-us/neural-compute-stick) installed in this node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub brain_tumor_seg_job.sh -l nodes=1:idc004nc2 -F "results/ncs2/ MYRIAD FP16" -N brain_ts_ncs2
print(job_id_ncs2[0])
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2', f'i_progress_{job_id_ncs2[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Vision Accelerator Design with Intel® Movidius™ VPUs
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te CPU](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on an [Intel® Vision Accelerator Design with Intel® Movidius™ VPUs](https://software.intel.com/openvino-toolkit/hardware#VPU) accelerator installed in this node that has eight VPUs.

In [ ]:
#Submit job to the queue
job_id_hddlr = !qsub brain_tumor_seg_job.sh -l nodes=1:idc002mx8 -F "results/hddlr/ HDDL FP16" -N brain_ts_hddlr
print(job_id_hddlr[0])
#Progress indicators
if job_id_hddlr:
    progressIndicator('results/hddlr', f'i_progress_{job_id_hddlr[0]}.txt', "Inference", 0, 100)

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

### View results

Once the jobs have completed, the queue system outputs the stdout and stderr streams of each job into files with names of the forms <*JobName*>.o<*JobID*> and <*JobName*>.e<*JobID*>, respecitvely.  The *JobName* corresponds to the `-N` option when submitting the job using the `qsub` command.  

The output image files for each job are written to the files `pred*.png` located in the directory `results/<device>` that was specified as the output directory to the job file.  We will now use the `outputHTML()` utility to display the output image files within this Jupyter* Notebook.  Calling `outputHTML()` from a Python* cell follows the form:
```python
outputHTML(title, directory, extension)
```
The parameters are:
- *title* - Title to put at the top of the displayed output
- *directory* - Directory containing output image files to display
- *extension* - File extension of output image files to display

Run the cells below to display the output images.


##### View results from an Intel® CPU

In [ ]:
outputHTML('Intel Core CPU', 
          'results/core', '.png')

##### View results from an Intel® Xeon® Gold 6258R CPU

In [ ]:
outputHTML('Intel Xeon CPU',
          'results/xeon_cascade_lake', '.png')

##### View results from an Intel® Xeon® E3-1268L v5 CPU

In [ ]:
outputHTML('Intel Xeon CPU',
          'results/xeon_skylake', '.png')

##### View results from an Intel® Core CPU and integrated Intel® GPU

In [ ]:
outputHTML('Intel Core + Intel GPU',
          'results/gpu', '.png')

##### View results from an Intel® Neural Compute Stick 2

In [ ]:
outputHTML('Intel CPU + Intel NCS2', 
          'results/ncs2', '.png')

##### View results from Intel® Vision Accelerator Design with Intel® Movidius™ VPUs

In [ ]:
outputHTML('Intel VPU HDDL-R',
          'results/hddlr', '.png')

### View and assess performance results

The running time of each inference task is recorded in `results/<device>/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time** and higher values mean better performance for **Inference Engine FPS**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

In [ ]:
# Create table of <architecture>, <title> for plotting
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('xeon_skylake', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nHD530/GPU'),
             ('ncs2', 'Intel VPU\ni5-6500TE\nNCS2 VPU'),
             ('hddlr', 'Intel VPU\n8xVPU\nHDDL-R')]
# For each archtecture in table, create path to stats file or placeholder 
stats_list = []
for arch, a_name in arch_list:
    # if job_id_<architecture> exists, the job was run and has a stats file
    if 'job_id_'+arch in vars():
        stats_list.append((f'results/{arch}/stats_{vars()["job_id_"+arch][0]}.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
# Plot the execution time from the stats files
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time')
# Plot the frames per second from the stats files
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps')

## Telemetry Dashboard
Once your submitted jobs are completed, run the cells below to view telemetry dashboards containing performance metrics for your model and target architecture.

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-6500TE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_cascade_lake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® E3-1268L v5 CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_skylake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Core CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_gpu[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_hddlr[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® NCS 2 (Neural Compute Stick 2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_ncs2[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>